In [2]:
!pip install mediapipe>=0.10.8  # For new API
!wget https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/latest/pose_landmarker_lite.task

--2025-12-17 11:07:51--  https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/latest/pose_landmarker_lite.task
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.207, 74.125.26.207, 172.217.204.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777746 (5.5M) [application/octet-stream]
Saving to: ‘pose_landmarker_lite.task’

pose_landmarker_lit 100%[===================>]   5.51M  --.-KB/s    in 0.1s    

2025-12-17 11:07:51 (50.6 MB/s) - ‘pose_landmarker_lite.task’ saved [5777746/5777746]



In [4]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pandas as pd
import numpy as np

# ---------------------------
# CONFIG
# ---------------------------
VIDEO_PATH = "/content/Shot2.mp4"
OUTPUT_CSV = "shot_02_biomech.csv"
OUTPUT_FRAME = "shot_02_skeleton_frame.jpg"

# ---------------------------
# Pose Connections for Drawing
# ---------------------------
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21),
    (17, 19), (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20),
    (11, 23), (12, 24), (23, 24), (23, 25), (24, 26), (25, 27), (26, 28),
    (27, 29), (28, 30), (29, 31), (30, 32), (27, 31), (28, 32)
]

# ---------------------------
# Draw Landmarks Function
# ---------------------------
def draw_landmarks(image, landmarks):
    """Draw pose landmarks and connections on image"""
    h, w, _ = image.shape

    # Draw connections
    for connection in POSE_CONNECTIONS:
        start_idx, end_idx = connection
        if start_idx < len(landmarks) and end_idx < len(landmarks):
            start = landmarks[start_idx]
            end = landmarks[end_idx]

            start_point = (int(start.x * w), int(start.y * h))
            end_point = (int(end.x * w), int(end.y * h))

            cv2.line(image, start_point, end_point, (0, 255, 0), 2)

    # Draw landmarks
    for lm in landmarks:
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(image, (cx, cy), 5, (0, 0, 255), -1)
        cv2.circle(image, (cx, cy), 7, (255, 255, 255), 2)

    return image

# ---------------------------
# MediaPipe Setup (New API)
# ---------------------------
base_options = python.BaseOptions(model_asset_path='pose_landmarker_lite.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=False,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5
)

detector = vision.PoseLandmarker.create_from_options(options)

# ---------------------------
# Video Load
# ---------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
data = []
frame_id = 0
saved_overlay = False

print("Processing video...")

# ---------------------------
# Process Video
# ---------------------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # Convert to MediaPipe Image format
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

    # Detect pose
    detection_result = detector.detect(mp_image)

    if detection_result.pose_landmarks:
        pose_landmarks = detection_result.pose_landmarks[0]  # First person

        row = {"frame": frame_id}

        for idx, lm in enumerate(pose_landmarks):
            row[f"lm_{idx}_x"] = lm.x
            row[f"lm_{idx}_y"] = lm.y
            row[f"lm_{idx}_z"] = lm.z
            row[f"lm_{idx}_visibility"] = lm.visibility

        data.append(row)

        # Save one skeleton overlay frame
        if not saved_overlay:
            annotated_frame = frame.copy()
            annotated_frame = draw_landmarks(annotated_frame, pose_landmarks)
            cv2.imwrite(OUTPUT_FRAME, annotated_frame)
            saved_overlay = True
            print(f"Saved overlay frame at frame {frame_id}")

    # Progress indicator
    if frame_id % 30 == 0:
        print(f"Processed {frame_id} frames...")

cap.release()
detector.close()

# ---------------------------
# Save CSV
# ---------------------------
df = pd.DataFrame(data)
df.to_csv(OUTPUT_CSV, index=False)

print("\n✅ Pose extraction complete")
print(f"Saved: {OUTPUT_CSV}")
print(f"Saved overlay frame: {OUTPUT_FRAME}")
print(f"Total frames processed: {frame_id}")
print(f"Frames with pose detected: {len(data)}")

Processing video...
Saved overlay frame at frame 1
Processed 30 frames...
Processed 60 frames...
Processed 90 frames...

✅ Pose extraction complete
Saved: shot_02_biomech.csv
Saved overlay frame: shot_02_skeleton_frame.jpg
Total frames processed: 98
Frames with pose detected: 98
